In [2]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")



#한글 시각화
from matplotlib import font_manager, rc




font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus'] = False

##### 2020년 9월뉴스 하나만 활용 [네이버 뉴스 크롤링데이터 ]

In [3]:

df = pd.read_csv('C:/Users/user654/PycharmProjects/NewsCrawling/2020_5~8/Article_경제_202008_202008.csv', engine = 'python', header = None)
df.columns = ['날짜','카테고리','언론사','제목','내용','URL']
df = df.drop_duplicates(['제목'])


df = df.reset_index()
df.drop(['index','URL'], axis = 1 , inplace = True)

df

,날짜,카테고리,언론사,제목,내용
0,20200801,경제,연합뉴스,계약갱신청구권은 위헌…헌재에 헌법소원심판 청구,정부 규제 반대하는 부동산 관련 단체 회원들 서울 연합뉴스 최재구 기자 1일 서울 ...
1,20200801,경제,SBS,의협 의대 정원 증원 등 중단 않으면 14일 총파업,대한의협 대정부 요구사항 발표 긴급 기자회견 대한의사협회가 의대 정원 증원 중단 등...
2,20200801,경제,뉴스1,부동산 규제 규탄하며 신발 던지는 시민들,서울 뉴스1 박지혜 기자 6·17 규제 소급적용 피해자 구제를 위한 모임 임대사업자...
3,20200801,경제,뉴스1,신발 던지는 부동산 규제 규탄 집회,서울 뉴스1 박지혜 기자 6·17 규제 소급적용 피해자 구제를 위한 모임 임대사업자...
4,20200801,경제,연합뉴스,모멘트 아무것도 없는 바닥에서 높은 꿈 펼칠 미래를 준비하는 이들을 위해,서울 연합뉴스 김도훈 기자 1일 오후 서울 청계천 광통교에서 인천공항공사 노조 주최...
...,...,...,...,...,...
32633,20200831,경제,로이터 증권(신),Norges Bank 2020년 8월 24일 현재 Tata Motors Ltd. 수...,8월 31일 로이터 Tata Motors Ltd NORGES BANK는 2020년 ...
32634,20200831,경제,로이터 증권(신),FDA Medtronic의 최소 770G 시스템 승인 2세에서 6세 사이의 사람들이...,8월 31일 로이터 메드트로닉 PLC FDA 제1형 당뇨병과 함께 2세에서 6세 사...
32635,20200831,경제,로이터 증권(신),사솔 허리케인 로라가 루이지애나 남서부의 사솔 호수 찰스 화학 단지 근처에 상륙했다.,8월 31일 로이터 Sasol Ltd 사솔 8월 27일 허리케인 로라가 루이지애나 ...
32636,20200831,경제,로이터 증권(신),바이오메리카 최고재무책임자 퇴직 신규 CFO 채용 이사진 선임 발표,8월 31일 로이터 바이오메리카 주식회사 .O BIOMERICA 최고재무책임자 CF...


##### 뉴스기사 내용 토큰화 및 동시에 어근 추출

In [4]:
from eunjeon import Mecab
import re

# mecab형태소 분석기에서 명사 추출
def mecabNoun(tokens):
    
    m = Mecab()
    
    tokens = re.sub(pattern = '[^ a-bA-Z가-힣]', repl = '', string = tokens)
    
    nounList = m.nouns(tokens)
    
    #2음절 이상 명사만 활용
    nounList = [i for i in nounList if len(i) >=2]
    
    return nounList

##### Doc2Vec

In [9]:
sentences = df['내용'].apply(lambda x : mecabNoun(x))
sentences = df['내용'].values

In [5]:
taggedDocs = [TaggedDocument(words=s, tags=[f"Article_{i}"]) for i, s in enumerate(sentences)]

taggedDocs[:3]

Wall time: 1min 52s


[TaggedDocument(words='정부 규제 반대하는 부동산 관련 단체 회원들 서울 연합뉴스 최재구 기자 1일 서울 여의도에서 617규제소급적용 피해자모임 임대사업자협회 추인위원회 등 부동산 관련단체 회원들이 정부의 부동산 규제에 반대하는 집회를 열고 임대차 3법 반대 등의 구호를 외치고 있다. 2020.8.1 jjaeck9 yna.co.kr 서울 연합뉴스 김철선 기자 사법시험준비생모임 사준모 은 계약갱신청구권제를 도입한 개정 주택임대차보호법이 임차인과 임대인의 기본권을 침해해 위헌 소지가 있다며 헌법재판소에 헌법소원 심판을 청구했다고 1일 밝혔다. 사준모는 계약갱신청구권제 도입으로 부동산 시장에 전세 물량이 급격하게 줄어 전셋집에 사는 임차인이 다른 전셋집으로 이전하기가 사실상 불가능해졌고 앞으로 전세 물량 감소와 전셋값 폭등이 예상된다 며 현재 좋은 전셋집에 사는 임차인에게만 이득을 줘 평등권을 침해한다 고 주장했다. 사준모는 또 계약갱신청구권제가 임대인의 계약의 자유와 재산권을 침해하고 헌법상 시장경제 질서의 원칙과 소급입법에 의한 재산권 침해 금지 원칙에도 반한다고 주장했다. 그러면서 정부·여당의 부동산 정책 실패 책임을 왜 국민이 떠안아야 하나 라며 헌재가 계약갱신요구권을 도입한 주택임대차보호법에 위헌 결정을 내려달라 고 요구했다. 계약갱신청구권 전월세상한제 도입을 골자로 하는 개정 주택임대차보호법은 지난달 29일 국회 법제사법위원회에 상정돼 통과된 데 이어 30일 국회 본회의에서 의결됐고 임시 국무회의를 거쳐 31일부터 시행됐다. 계약갱신청구권 도입으로 세입자는 추가 2년의 계약 연장을 요구할 수 있고 집주인은 실거주 등의 사정이 없으면 이를 받아들여야 한다. 전월세상한제는 계약 갱신 시 임대료 상승폭을 5% 이내로 제한하는 내용이다.', tags=['Article_0']),
 TaggedDocument(words='대한의협 대정부 요구사항 발표 긴급 기자회견 대한의사협회가 의대 정원 증원 중단 등을 요구하며 정부가 의료계의 요구사항을 수용하

In [6]:
%%time

DVmodel = Doc2Vec(documents = taggedDocs, 
                  vector_size = 200)

Wall time: 1min 14s


In [74]:
selectArticle = 'Article_5'
rowList_ = [int(i[0].split('_')[1]) for i in DVmodel.docvecs.most_similar(selectArticle)]

sim_df = df.loc[rowList_ , :'내용']
sim_df['코사인 유사도'] = [i[1] for i in DVmodel.docvecs.most_similar(selectArticle)]
sim_df

,날짜,카테고리,언론사,제목,내용,코사인 유사도
57,20200801,경제,연합뉴스TV,7월 수출 7%↓…코로나 이후 감소폭 첫 한자릿수,앵커 수출이 넉달만에 한자릿수 감소세로 돌아왔습니다. 수출규모 역시 400억달러대를...,0.795211
89,20200801,경제,노컷뉴스,7월 수출 7.0%↓…코로나 쇼크 회복세,15개 품목 중 6개 플러스 …바이오·헬스 컴퓨터 수출 호조 CBS노컷뉴스 김선경 ...,0.725041
99,20200801,경제,서울경제,韓 수출 회복 시그널 7월 수출 7.0%↓···4개월만에 한 자릿수대 감소,바이오·헬스 컴퓨터 수출 호조세 산업부 “여러 면에서 긍정적 회복 신호” 연합뉴스 ...,0.724069
105,20200801,경제,파이낸셜뉴스,7월 수출 7% 감소…코로나19 이후 4개월 만에 한 자릿수대 감소,사진 뉴스1 파이낸셜뉴스 신종 코로나바이러스 감염증 코로나19 확산 여파에도 지난달...,0.702612
273,20200802,경제,문화일보,7월 수출 7.0%…5개 월 연속 마이너스,코로나 사태 본격화 후 첫 한 자릿수 감소 지난달 우리나라 수출이 신종 코로나바이러...,0.701724
77,20200801,경제,KBS,7월 수출 7.0%↓…넉 달 만에 한 자릿수대 감소,수출이 넉 달 만에 한 자릿수대 감소를 기록했습니다. 산업통상자원부가 오늘 1일 발...,0.695756
111,20200801,경제,연합뉴스,2보 7월 수출 7.0%↓ 넉 달 만에 한 자릿수대 감소,4개월만에 수출규모 400억 달러대…15대 품목 중 6개 수출 증가 무역수지 3개월...,0.691668
11429,20200811,경제,YTN,8월 초 수출 87억 달러...1년 전보다 23.6% 감소,8월 1일부터 10일까지 수출이 1년 전보다 23.6% 감소한 87억 달러로 집계됐...,0.678900
87,20200801,경제,이데일리,5개월째 수출 마이너스지만 감소폭 뚝…400억달러대 회복 종합,7월 수출 7% 감소..감소폭 한자릿수로 축소 반도체 5.6% 늘고 IT 바이오헬스...,0.674198
1024,20200803,경제,매일경제,쌍용차 7월 국내외서 7489대 판매…전년比 30.6%↓,쌍용자동차가 지난 7월 내수 6702대 수출 787대를 포함해 총 7489대를 판매...,0.671093


In [10]:
targetRow = 600 #임의의 target 설정


rowList = [targetRow]
similarTartgetList = DVmodel.docvecs.most_similar('Article_' + str(targetRow), topn = 10)


for i in similarTartgetList:
    rowList.append(int(i[0].split('_')[1]))

    
targetDF = df.loc[rowList, :]
targetDF['Similarity'] = [1] + [i[1] for i in similarTartgetList]
targetDF

,날짜,카테고리,언론사,제목,내용,Similarity
600,20200802,경제,헤럴드경제,“LG화학 주가 80만원 간다”,하나금융투자 LG화학 목표주가 상향 NCC 경쟁력 지속 화학 사업 호실적 지속 흑자...,1.000000
1589,20200803,경제,조선비즈,삼성중공업 2분기 어닝쇼크에 급락세,삼성중공업 010140 이 2분기 ‘어닝쇼크’를 기록했다는 소식에 급락세다. 3일 ...,0.628752
32381,20200831,경제,머니투데이,나스미디어 KT 빅데이터 활용 기대감…목표가↑신한금투,머니투데이 구유나 기자 신한금융투자는 31일 나스미디어에 대해 KT 생태계 구축의 ...,0.627879
4960,20200805,경제,머니투데이,코스맥스엔비티 선제적 해외진출 효과 톡톡하나,머니투데이 김태현 기자 하나금융투자는 코스맥스엔비티에 대해 선제적 해외 진출로 향후...,0.612302
29066,20200827,경제,이데일리,특징주‘목표주가 100만원’ LG화학 ‘강세’…52주 최고가,이데일리 유준하 기자 LG화학 051910 이 석유화학 전 사업부문 호실적을 달성하...,0.603665
20375,20200819,경제,파이낸셜뉴스,삼성화재 수익성 개선 추세 지속 이베스트투자증권,파이낸셜뉴스 이베스트투자증권은 19일 삼성화재에 대해 수익성 개선 추세가 지속될 것...,0.600175
20160,20200819,경제,매일경제,특징주 삼성화재 2분기 호실적에 강세,삼성화재가 2분기 호실적에 장 초반 강세다. 19일 오전 9시32분 현재 삼성화재는...,0.597268
1370,20200803,경제,헤럴드경제,itM특징주카카오 2분기 실적 발표 앞두고 주가 상승세,6일 2분기 실적 발표 앞두고 상승폭 확대 헤럴드경제 이태형 기자 올 2분기 사상 ...,0.595403
5209,20200805,경제,로이터 증권(신),매슨 주당 2분기 수익 0.76달러 기록,8월 5일 로이터 맷슨 주식회사 .N MATSON Inc. 2 4분기 2020년 결...,0.593720
23815,20200823,경제,YTN,국내 외국기업 지난해 수익 늘었지만 투자·고용은 감소,지난해 외국계 기업들은 국내 매출과 수익이 늘었지만 평균 투자와 고용은 감소한 것으...,0.592803


In [11]:
DVmodel.docvecs.most_similar('Article_' + str(targetRow))

[('Article_1589', 0.6287522315979004),
 ('Article_32381', 0.6278785467147827),
 ('Article_4960', 0.6123019456863403),
 ('Article_29066', 0.6036648750305176),
 ('Article_20375', 0.6001745462417603),
 ('Article_20160', 0.5972684025764465),
 ('Article_1370', 0.595402717590332),
 ('Article_5209', 0.5937200784683228),
 ('Article_23815', 0.5928031206130981),
 ('Article_3802', 0.5917827486991882)]

In [40]:
a = np.round(pd.DataFrame(DVmodel.docvecs.doctag_syn0), 3)
a.index = ['Article{}'.format(i) for i in range(df.shape[0])]
a

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).
  """Entry point for launching an IPython kernel.


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
Article0,0.490,-0.062,0.614,0.077,-0.030,-0.194,-0.136,0.142,-0.020,0.317,...,-0.477,-0.014,-0.132,-0.387,0.231,0.076,-0.359,-0.190,-0.175,-0.311
Article1,0.300,0.112,0.297,-0.103,-0.129,-0.032,0.167,-0.280,0.249,-0.053,...,-0.310,-0.054,-0.075,-0.134,0.217,0.054,-0.037,-0.180,0.164,-0.008
Article2,0.270,-0.128,0.297,-0.041,-0.073,-0.055,0.207,-0.095,0.178,0.192,...,-0.345,-0.015,-0.035,-0.272,0.071,0.020,-0.027,-0.043,-0.013,-0.092
Article3,0.241,-0.144,0.197,-0.067,-0.141,0.025,0.131,-0.085,0.060,0.120,...,-0.267,-0.005,0.072,-0.320,0.125,0.089,-0.005,-0.121,0.029,-0.090
Article4,0.209,0.034,0.229,0.129,-0.177,0.028,0.032,0.002,0.037,0.197,...,-0.213,-0.146,0.027,-0.059,0.067,-0.173,-0.068,-0.187,0.011,-0.006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Article32633,0.140,0.023,0.168,-0.007,-0.074,-0.038,0.057,0.029,-0.041,0.147,...,-0.225,-0.041,-0.049,0.040,0.025,-0.021,-0.086,-0.047,0.043,-0.042
Article32634,0.034,-0.044,0.246,-0.042,0.008,-0.173,0.075,0.054,-0.076,0.205,...,0.153,-0.115,0.033,0.056,-0.224,0.053,-0.104,-0.283,0.395,-0.062
Article32635,0.241,-0.446,0.335,0.197,-0.068,0.455,0.002,0.292,0.049,0.352,...,-0.009,-0.222,0.140,0.190,-0.314,-0.150,0.025,0.125,-0.188,0.015
Article32636,0.136,0.039,0.186,-0.250,0.095,0.035,0.072,0.191,0.115,0.167,...,-0.371,-0.011,-0.147,-0.218,0.229,-0.343,0.159,-0.138,-0.077,0.178
